In [69]:
import pandas as pd
from sqlalchemy import create_engine

# Put postgreSQL credeitials into sql_login.py
from sql_login import USERNAME as USERNAME
from sql_login import PASSWORD as PASSWORD

## Read from CSV

In [70]:
avocado_df = pd.read_csv('Resources/Avocado.csv')

In [71]:
avocado_df

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27318,2015-11-22,1.97,6249.43,1733.40,2873.92,30.95,1611.16,1590.00,21.16,0.0,organic,2015,WestTexNewMexico
27319,2015-11-29,2.08,4638.10,1395.02,2238.04,61.71,943.33,943.33,0.00,0.0,organic,2015,WestTexNewMexico
27320,2015-12-13,1.80,7836.65,2194.49,2981.01,25.97,2635.18,2598.45,36.73,0.0,organic,2015,WestTexNewMexico
27321,2015-12-20,1.92,6255.19,1512.45,2407.32,11.78,2323.64,2213.72,109.92,0.0,organic,2015,WestTexNewMexico


## Data Munging

In [72]:
# Get sales for volume sold on date
avocado_df['Total Sales'] = avocado_df['AveragePrice']*avocado_df['Total Volume']

In [73]:
# Drop unnecessary columns
avocado_df = avocado_df[['Date','Total Volume','Total Sales']]

# Sum total sales and volume on dates for all locations, not differentiating conventional/organic
avocado_gb = avocado_df.groupby(['Date'])
avocado_date_df = pd.DataFrame(avocado_gb.sum().reset_index())

avocado_date_df['price'] = avocado_date_df['Total Sales']/avocado_date_df['Total Volume']

# Extract year and quarter from date
a_date_df_len = avocado_date_df.shape[0]
year_list = [int(avocado_date_df.iloc[j,0][0:4]) for j in range(0,a_date_df_len)]
quarter_list = [(int(avocado_date_df.iloc[j,0][5:7])-1)//3+1 for j in range(0,a_date_df_len)]
id_list = [j for j in range(0,a_date_df_len)]

avocado_date_df['id'] = id_list
avocado_date_df['year'] = year_list
avocado_date_df['quarter'] = quarter_list

#for j in range(1,13):
#    print(j,(j-1)//3+1)

# Rearrange and rename columns
avocado_date_df = avocado_date_df[['id','year','quarter','Date','price']].rename(columns={'Date':'date'})

avocado_date_df

,id,year,quarter,date,price
0,0,2015,1,2015-01-04,0.957502
1,1,2015,1,2015-01-11,1.019967
2,2,2015,1,2015-01-18,1.044620
3,3,2015,1,2015-01-25,1.052524
4,4,2015,1,2015-02-01,0.902667
...,...,...,...,...,...
248,248,2019,4,2019-11-03,1.061531
249,249,2019,4,2019-11-10,1.035006
250,250,2019,4,2019-11-17,1.042074
251,251,2019,4,2019-11-24,1.081343


## Connect to postgreSQL and import the dataframe

In [74]:
rds_connection_string = USERNAME+':'+PASSWORD+"@localhost:5432/avocados"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.engine.connect()

In [75]:
# Drop avocado price table if it exists
conn.execute('DROP TABLE IF EXISTS avo_price;')

In [76]:
# Create table to hold avocado price data
conn.execute(
"""
    CREATE TABLE avo_price (
    id INT PRIMARY KEY,
    year INT,
    quarter INT,
    date date,
    price double precision);
"""
)

In [77]:
# Confirm tables
engine.table_names()

['cmg', 'avo_price']

In [78]:
avocado_date_df[['id','year','quarter','date','price']].to_sql(name='avo_price', con=engine, if_exists='append', index=False)